# Advent of Code 2020
## All Answers, Timed

In [148]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
from functools import reduce

In [113]:
def day_1a(df):
    df_matches = df.merge(2020 - df)
    return df_matches.value[0] * df_matches.value[1]

def day_1b(df):
    df_sums = 2020 - df
    df_b = pd.DataFrame(
        data=df_sums.apply(lambda s: (s - df).value, axis='columns').values.flatten(),
        columns=['value']
    ).drop_duplicates()
    df_c = pd.DataFrame(
        data=df_b.apply(lambda b: (df_sums - b).value, axis='columns').values.flatten(),
        columns=['value']
    ).drop_duplicates()
    df_results = df.merge(df_b).merge(df_c)
    return df_results.value[0] * df_results.value[1] * df_results.value[2]

def day_1():
    df = pd.read_csv('./data/day1.txt', names=['value'])
    return (day_1a(df), day_1b(df))

In [114]:
def meets_day_2a_criteria(record):
    if match := re.match(r'(\d+)-(\d+) ([A-Za-z]{1}): (.+)', record):
        min_chars = int(match.group(1))
        max_chars = int(match.group(2))
        char = match.group(3)
        pw = match.group(4)
        char_count = np.char.count(pw, char)
        return char_count <= max_chars and char_count >= min_chars
    else:
        return False

def meets_day_2b_criteria(record):
    if match := re.match(r'(\d+)-(\d+) ([A-Za-z]{1}): (.+)', record):
        pos_1 = int(match.group(1))
        pos_2 = int(match.group(2))
        char = match.group(3)
        pw = list(match.group(4))
        pw_count = len(pw)
        char_1 = '' if pw_count < pos_1 else pw[pos_1 - 1]
        char_2 = '' if pw_count < pos_2 else pw[pos_2 - 1]
        char_count = np.char.count([char_1, char_2], char).sum()
        return char_count == 1
    else:
        return False

def day_2a(df):
    df_validation = df.record.map(meets_day_2a_criteria).to_frame()
    return df[df_validation].dropna().count().values[0]

def day_2b(df):
    df_validation_2 = df.record.map(meets_day_2b_criteria).to_frame()
    return df[df_validation_2].dropna().count().values[0]

def day_2():
    df = pd.DataFrame(
        data=open('./data/day2.txt', 'r').readlines(),
        columns=['record']
    ).record.str.strip().to_frame()
    return (day_2a(df), day_2b(df))

In [115]:
def extract_char(pattern, index):
    length = len(pattern)
    return pattern[index if index < length else index % length]

def count_trees(df, steps):
    df_x = df.index.map(lambda i: extract_char(df.terrain[i], i * steps)).to_frame(index=False)
    return df_x[df_x[0] == '#'].count().values[0]

def day_3b(df):
    return np.array([
        count_trees(df, 1),
        count_trees(df, 3),
        count_trees(df, 5),
        count_trees(df, 7),
        count_trees(df[df.index % 2 == 0].reset_index(drop=True), 1)
    ]).prod()

def day_3():
    df = pd.read_csv('./data/day3.txt', names=['terrain'])
    return (count_trees(df, 3), day_3b(df))

In [204]:
def validate_pattern(pattern, f_test):
    def validate(text):
        if match := re.findall(pattern, text):
            return f_test(*match[0])
        else:
            return False
    return validate

def validate_height(value, uom):
    if uom == 'cm':
        return 150 <= value <= 193
    elif uom == 'in':
        return 59 <= value <= 76
    else:
        return False

def validate(passport):
    b_results = map(lambda f: f(passport), [
        validate_pattern(r'(byr):(\d+)', lambda _, d: 1920 <= int(d) <= 2002),
        validate_pattern(r'(iyr):(\d+)', lambda _, d: 2010 <= int(d) <= 2020),
        validate_pattern(r'(eyr):(\d+)', lambda _, d: 2020 <= int(d) <= 2030),
        validate_pattern(r'(hgt):(\d+)([^\s]+)', lambda _, d, t: validate_height(int(d), t)),
        validate_pattern(r'(hcl):(#[0-9a-f]{6})', lambda _, v: True),
        validate_pattern(r'(ecl):([^\s]+)', lambda _, v: v in ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']),
        validate_pattern(r'(pid):(\d{9})', lambda _, v: True)
    ])
    return reduce(lambda l, r: l and r, b_results, True)

def day_4():
    s_input = Path('./data/day4.txt').read_text().strip()
    df = pd.DataFrame(data=re.split(r'^$', s_input, flags=re.MULTILINE))
    df_x = df[0].map(lambda e: len([x for x in re.findall(r'(\w+):[^\s]+', e) if x != 'cid']) == 7).to_frame()
    df_min_props = df[df_x].dropna()
    df_valid_props = df_min_props[df_min_props[0].map(validate).to_frame()].dropna()
    return (df_min_props.count().values[0], df_valid_props.count().values[0])

In [205]:
%%time
print('Day 1:', day_1())
print('Day 2:', day_2())
print('Day 3:', day_3())
print('Day 4:', day_4())

Day 1: (969024, 230057040)
Day 2: (418, 616)
Day 3: (176, 5872458240)
Day 4: (264, 225)
CPU times: user 770 ms, sys: 9.38 ms, total: 779 ms
Wall time: 792 ms


In [199]:
i = f'''iyr:2015 cid:189 ecl:oth byr:1947 hcl:#6c4ab1 eyr:2026
hgt:174cm
pid:526744288'''
r = r'(eyr):([^\s]+)'
m = re.findall(r, i)

def do(_, a):
    print('a is', a)

l = ['eyr', 'byr', 'ecl']
x = 'byr' in l
x

True